In [2]:
# Análise de Média de Pessoas em Aglomerações por Período - POR REGIÃO
# Base: Planilha RAW (data/raw)
# Data início: Julho/2025 (01/07/2025 - fixo)
# Data fim: Informada pelo usuário
# Períodos: Manhã (10h) e Tarde (15h)

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Definir caminho da raiz do projeto
# Funciona tanto como script .py quanto em notebook
if '__file__' in globals():
    # Executando como script .py
    projeto_root = Path(__file__).parent.parent.parent
else:
    # Executando como notebook ou célula
    projeto_root = Path.cwd()
    # Se estiver em notebooks/, subir um nível
    if projeto_root.name == 'notebooks':
        projeto_root = projeto_root.parent

print("=" * 80)
print("ANÁLISE: Média de Pessoas em Aglomerações por Período (POR REGIÃO)")
print("=" * 80)
print(f"✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Caminhos

# %%
RAW_DIR = projeto_root / 'data' / 'raw'
DOCS_DIR = projeto_root / 'docs'

# Garantir que o diretório docs existe
DOCS_DIR.mkdir(exist_ok=True)

print(f"\n✓ Diretório base: {projeto_root}")
print(f"✓ Planilhas RAW: {RAW_DIR}")
print(f"✓ Saída de documentos: {DOCS_DIR}")

# %% [markdown]
# # 3. Listar e Selecionar Planilha RAW

# %%
# Listar arquivos .xlsx na pasta raw
arquivos_disponiveis = sorted(RAW_DIR.glob('*.xlsx'))

if not arquivos_disponiveis:
    print("\n⚠️  ERRO: Nenhum arquivo .xlsx encontrado na pasta 'data/raw/'")
    print("   Certifique-se de que existem planilhas na pasta raw.")
    exit()

print(f"\n📁 Arquivos disponíveis em 'data/raw/':")
print("-" * 80)
for idx, arquivo in enumerate(arquivos_disponiveis, 1):
    print(f"  [{idx}] {arquivo.name}")
print("-" * 80)

# Solicitar seleção do usuário
while True:
    try:
        selecao = input(f"\nSelecione o número do arquivo [1-{len(arquivos_disponiveis)}]: ").strip()
        idx_selecionado = int(selecao) - 1
        
        if 0 <= idx_selecionado < len(arquivos_disponiveis):
            arquivo_selecionado = arquivos_disponiveis[idx_selecionado]
            print(f"\n✓ Arquivo selecionado: {arquivo_selecionado.name}")
            break
        else:
            print(f"⚠️  Por favor, escolha um número entre 1 e {len(arquivos_disponiveis)}")
    except ValueError:
        print("⚠️  Entrada inválida. Digite apenas o número do arquivo.")
    except KeyboardInterrupt:
        print("\n\n⚠️  Operação cancelada pelo usuário.")
        exit()

# %% [markdown]
# # 4. Solicitar Data Fim da Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DE ANÁLISE")
print("=" * 80)
print("\n📅 Data de início: 01/07/2025 (fixo)")

while True:
    try:
        data_fim_input = input("\n📅 Digite a data FIM da análise (formato: DD/MM/AAAA): ").strip()
        data_fim = datetime.strptime(data_fim_input, '%d/%m/%Y')
        
        # Validar se a data fim é posterior a julho de 2025
        data_inicio = datetime(2025, 7, 1)
        if data_fim < data_inicio:
            print("⚠️  A data fim deve ser posterior a 01/07/2025")
            continue
        
        print(f"\n✓ Data fim definida: {data_fim.strftime('%d/%m/%Y')}")
        print(f"✓ Período de análise: 01/07/2025 até {data_fim.strftime('%d/%m/%Y')}")
        break
        
    except ValueError:
        print("⚠️  Formato inválido. Use DD/MM/AAAA (exemplo: 20/10/2025)")
    except KeyboardInterrupt:
        print("\n\n⚠️  Operação cancelada pelo usuário.")
        exit()

# %% [markdown]
# # 5. Carregar e Validar Dados

# %%
print(f"\n📊 Carregando dados de: {arquivo_selecionado.name}")
print("-" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Planilha carregada com sucesso!")
    print(f"✓ Total de registros: {len(df):,}")
    
except Exception as e:
    print(f"\n⚠️  ERRO ao carregar planilha: {e}")
    exit()

# %% [markdown]
# # 6. Validar Estrutura Esperada

# %%
colunas_esperadas = ['Data', 'Equipe', 'Logradouro', 'Período', 'Quantidade']

colunas_faltantes = [col for col in colunas_esperadas if col not in df.columns]

if colunas_faltantes:
    print(f"\n⚠️  ERRO: Colunas obrigatórias não encontradas:")
    for col in colunas_faltantes:
        print(f"   - {col}")
    print("\n   Certifique-se de que está usando uma planilha RAW.")
    exit()

print(f"\n✓ Estrutura da planilha validada!")

# %% [markdown]
# # 7. Extrair Região do Logradouro

# %%
# Extrair região (tudo antes do primeiro " - ")
df['Região'] = df['Logradouro'].str.split(' - ', n=1).str[0].str.strip()

print(f"\n🗺️  Extraindo regiões dos logradouros:")
print("-" * 80)
regioes_unicas = sorted(df['Região'].unique())
for regiao in regioes_unicas:
    qtd_log = df[df['Região'] == regiao]['Logradouro'].nunique()
    print(f"  • {regiao} ({qtd_log} logradouros)")

# %% [markdown]
# # 8. Converter Campo Data e Filtrar Período

# %%
# Converter coluna Data para datetime
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Verificar conversão
datas_invalidas = df['Data'].isna().sum()
if datas_invalidas > 0:
    print(f"\n⚠️  Atenção: {datas_invalidas} registros com datas inválidas (serão removidos)")
    df = df.dropna(subset=['Data'])

# Extrair mês e ano
df['Mes'] = df['Data'].dt.month
df['Ano'] = df['Data'].dt.year

print(f"\n✓ Conversão de datas concluída")
print(f"✓ Período dos dados: {df['Data'].min().strftime('%d/%m/%Y')} até {df['Data'].max().strftime('%d/%m/%Y')}")

# Filtrar dados de acordo com o período definido
data_inicio = datetime(2025, 7, 1)
df_filtrado = df[(df['Data'] >= data_inicio) & (df['Data'] <= data_fim)].copy()

print(f"\n🔍 Filtrando dados:")
print("-" * 80)
print(f"✓ Data início: {data_inicio.strftime('%d/%m/%Y')}")
print(f"✓ Data fim: {data_fim.strftime('%d/%m/%Y')}")
print(f"✓ Total de registros após filtro: {len(df_filtrado):,}")

if len(df_filtrado) == 0:
    print(f"\n⚠️  ERRO: Nenhum registro encontrado para o período especificado.")
    exit()

# %% [markdown]
# # 9. Garantir Tipo Numérico

# %%
# Garantir que Quantidade é numérico
df_filtrado['Quantidade'] = pd.to_numeric(df_filtrado['Quantidade'], errors='coerce')
df_filtrado = df_filtrado.dropna(subset=['Quantidade'])

print(f"\n✓ Total de registros com Quantidade válida: {len(df_filtrado):,}")

# %% [markdown]
# # 10. Mapear Períodos

# %%
# Mapear períodos para categorias
mapeamento_periodo = {
    '10h': 'Manhã',
    '15h': 'Tarde'
}

df_filtrado['Periodo_Categoria'] = df_filtrado['Período'].map(mapeamento_periodo)

# Verificar mapeamento
periodos_nao_mapeados = df_filtrado['Periodo_Categoria'].isna().sum()
if periodos_nao_mapeados > 0:
    print(f"\n⚠️  Atenção: {periodos_nao_mapeados} registros com períodos não mapeados (serão removidos)")
    df_filtrado = df_filtrado.dropna(subset=['Periodo_Categoria'])
else:
    print(f"\n✓ Todos os períodos mapeados com sucesso!")

print(f"\nPeríodos encontrados:")
for periodo in sorted(df_filtrado['Periodo_Categoria'].unique()):
    print(f"  • {periodo}")

# %% [markdown]
# # 11. Calcular Soma Total por Região (para ordenação)

# %%
print(f"\n📊 Calculando soma total por região:")
print("-" * 80)

# Calcular soma total de pessoas por região
soma_por_regiao = df_filtrado.groupby('Região')['Quantidade'].sum().sort_values(ascending=False)

print(f"✓ Regiões ordenadas por soma total:")
for regiao, soma in soma_por_regiao.items():
    print(f"  • {regiao}: {soma:,.0f} pessoas")

# Lista ordenada de regiões (será usada para ordenar as abas)
regioes_ordenadas = soma_por_regiao.index.tolist()

# Dicionário de nomes de meses
nomes_meses = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

# Meses únicos no dataset
meses_unicos = sorted(df_filtrado['Mes'].unique())

print(f"\n📅 Meses encontrados:")
for mes in meses_unicos:
    print(f"  • {nomes_meses[mes]}")

# %% [markdown]
# # 12. Criar Abas por Região (Nova Estrutura)

# %%
print(f"\n📊 Criando abas por região (novo formato):")
print("-" * 80)

periodos = ['Manhã', 'Tarde']

# Dicionários para armazenar DataFrames
dfs_regioes = {}
dfs_graficos = {}

for regiao in regioes_ordenadas:
    print(f"\nProcessando região: {regiao}")
    
    # Filtrar dados da região
    df_regiao = df_filtrado[df_filtrado['Região'] == regiao].copy()
    
    # Agrupar por Mês, Logradouro e Período
    agg_logradouro = df_regiao.groupby(['Mes', 'Logradouro', 'Periodo_Categoria']).agg({
        'Quantidade': 'sum',
        'Data': 'nunique'  # Contar dias únicos
    }).reset_index()
    agg_logradouro.rename(columns={'Data': 'Dias_Unicos'}, inplace=True)
    
    # Lista para armazenar linhas da aba principal
    linhas_regiao = []
    
    # Lista para armazenar linhas totais (para aba gráfico)
    linhas_grafico = []
    
    # Processar cada mês
    for mes in meses_unicos:
        # Filtrar dados do mês
        dados_mes = agg_logradouro[agg_logradouro['Mes'] == mes].copy()
        
        # Obter logradouros únicos do mês e ordenar alfabeticamente
        logradouros_mes = sorted(dados_mes['Logradouro'].unique())
        
        # Variáveis para acumular totais do mês
        soma_total_mes = 0
        soma_manha_mes = 0
        soma_tarde_mes = 0
        soma_dias_mes = 0  # Soma dos Qtd. Dias
        
        # Processar cada logradouro
        for logradouro in logradouros_mes:
            dados_log = dados_mes[dados_mes['Logradouro'] == logradouro]
            
            # Calcular soma por período
            soma_manha = dados_log[dados_log['Periodo_Categoria'] == 'Manhã']['Quantidade'].sum()
            soma_tarde = dados_log[dados_log['Periodo_Categoria'] == 'Tarde']['Quantidade'].sum()
            soma_total = soma_manha + soma_tarde
            
            # Contar dias únicos (considerar ambos os períodos)
            dias_manha = dados_log[dados_log['Periodo_Categoria'] == 'Manhã']['Dias_Unicos'].sum()
            dias_tarde = dados_log[dados_log['Periodo_Categoria'] == 'Tarde']['Dias_Unicos'].sum()
            # Usar o máximo entre os dois (caso um período tenha mais dias que o outro)
            dias_unicos = max(dias_manha, dias_tarde)
            
            # Acumular para total do mês
            soma_total_mes += soma_total
            soma_manha_mes += soma_manha
            soma_tarde_mes += soma_tarde
            soma_dias_mes += dias_unicos  # SOMA dos dias
            
            # Calcular médias
            media_total = soma_total / (dias_unicos * 2) if dias_unicos > 0 else 0
            media_manha = soma_manha / dias_unicos if dias_unicos > 0 else 0
            media_tarde = soma_tarde / dias_unicos if dias_unicos > 0 else 0
            
            # Adicionar linha do logradouro
            linhas_regiao.append({
                'Mês': nomes_meses[mes],
                'Logradouro': logradouro,
                'Soma Pessoas': int(soma_total),
                'Qtd. Dias': dias_unicos,
                'Média Pessoas': round(media_total, 2),
                'Média Manhã': round(media_manha, 2),
                'Média Tarde': round(media_tarde, 2)
            })
        
        # Calcular médias do TOTAL DO MÊS (usando SOMA dos dias)
        media_total_mes = soma_total_mes / (soma_dias_mes * 2) if soma_dias_mes > 0 else 0
        media_manha_mes = soma_manha_mes / soma_dias_mes if soma_dias_mes > 0 else 0
        media_tarde_mes = soma_tarde_mes / soma_dias_mes if soma_dias_mes > 0 else 0
        
        # Adicionar linha TOTAL DO MÊS
        linhas_regiao.append({
            'Mês': nomes_meses[mes].upper(),
            'Logradouro': 'TOTAL DO MÊS',
            'Soma Pessoas': int(soma_total_mes),
            'Qtd. Dias': soma_dias_mes,  # SOMA dos Qtd. Dias
            'Média Pessoas': round(media_total_mes, 2),
            'Média Manhã': round(media_manha_mes, 2),
            'Média Tarde': round(media_tarde_mes, 2)
        })
        
        # Adicionar linha para aba gráfico (apenas totais)
        linhas_grafico.append({
            'Mês': nomes_meses[mes],
            'Média Manhã': round(media_manha_mes, 2),
            'Média Tarde': round(media_tarde_mes, 2)
        })
    
    # Criar DataFrames
    dfs_regioes[regiao] = pd.DataFrame(linhas_regiao)
    dfs_graficos[regiao] = pd.DataFrame(linhas_grafico)
    
    print(f"  ✓ Aba '{regiao}' criada com {len(linhas_regiao)} linhas")
    print(f"  ✓ Aba '{regiao} - Gráfico' criada com {len(linhas_grafico)} linhas")

# %% [markdown]
# # 13. Exportar para Excel

# %%
nome_arquivo_saida = f"media_pessoas_por_regiao_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
caminho_saida = DOCS_DIR / nome_arquivo_saida

print(f"\n💾 Exportando para Excel:")
print("-" * 80)

try:
    with pd.ExcelWriter(caminho_saida, engine='openpyxl') as writer:
        # Para cada região (em ordem de soma total)
        for regiao in regioes_ordenadas:
            # Aba principal da região
            nome_aba = regiao
            if len(nome_aba) > 31:
                nome_aba = nome_aba[:31]
            
            dfs_regioes[regiao].to_excel(writer, sheet_name=nome_aba, index=False)
            print(f"  ✓ Aba '{nome_aba}' exportada")
            
            # Aba para gráfico da região
            nome_aba_grafico = f"{regiao} - Gráfico"
            if len(nome_aba_grafico) > 31:
                nome_aba_grafico = f"{regiao[:20]} - Gráfico"
            
            dfs_graficos[regiao].to_excel(writer, sheet_name=nome_aba_grafico, index=False)
            print(f"  ✓ Aba '{nome_aba_grafico}' exportada")
        
        # Ajustar largura das colunas em todas as abas
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = min(max_length + 2, 50)
                worksheet.column_dimensions[column].width = adjusted_width
    
    print(f"\n✓ Arquivo exportado com sucesso!")
    print(f"✓ Localização: {caminho_saida}")
    print(f"✓ Nome do arquivo: {nome_arquivo_saida}")
    
except Exception as e:
    print(f"\n⚠️  ERRO ao exportar arquivo: {e}")

# %% [markdown]
# # 14. Resumo Executivo

# %%
print(f"\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)

print(f"\n📊 Análise Concluída:")
print(f"  • Período: 01/07/2025 até {data_fim.strftime('%d/%m/%Y')}")
print(f"  • Total de registros: {len(df_filtrado):,}")
print(f"  • Meses analisados: {len(meses_unicos)}")
print(f"  • Regiões encontradas: {len(regioes_ordenadas)}")

print(f"\n📁 Arquivo gerado:")
print(f"  • {nome_arquivo_saida}")
print(f"  • Localização: docs/")

print(f"\n🗺️  Regiões processadas (ordenadas por soma total):")
for idx, regiao in enumerate(regioes_ordenadas, 1):
    print(f"  {idx}. {regiao} - Soma: {soma_por_regiao[regiao]:,.0f} pessoas")

print(f"\n📋 Estrutura das abas:")
print(f"  Para cada região:")
print(f"    • Aba principal: Logradouros por mês + linha TOTAL DO MÊS")
print(f"    • Aba gráfico: Apenas totais mensais (Manhã e Tarde)")

print(f"\n✅ Análise concluída com sucesso!")
print("=" * 80)

ANÁLISE: Média de Pessoas em Aglomerações por Período (POR REGIÃO)
✓ Bibliotecas importadas
✓ Análise iniciada em: 06/11/2025 17:28:37

✓ Diretório base: c:\Users\x504693\Documents\projetos\projeto_etl_dados
✓ Planilhas RAW: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\raw
✓ Saída de documentos: c:\Users\x504693\Documents\projetos\projeto_etl_dados\docs

📁 Arquivos disponíveis em 'data/raw/':
--------------------------------------------------------------------------------
  [1] Base Okuhara Kohei.xlsx
  [2] Base Tablet.xlsx
  [3] CidadaosVinculadosXBeneficios.xlsx
  [4] CONTAGEM 2025 - CnR.xlsx
  [5] Contagem diária - Compilado.xlsx
  [6] Contagem_diaria_centro - Padronizada.xlsx
  [7] nomes_Abordados.xlsx
  [8] RelatorioCidadaoVinculado.xlsx
--------------------------------------------------------------------------------

✓ Arquivo selecionado: CONTAGEM 2025 - CnR.xlsx

DEFINIR PERÍODO DE ANÁLISE

📅 Data de início: 01/07/2025 (fixo)

✓ Data fim definida: 05/11/2025
✓ Per